1. AutoML built on top of sklearn and optuna
2. Abstract and include mlflow as part of it. Mlflow should be storing directly in AWS S3
3. [Opt] Add code to include memory profiling of the code
4.  

In [2]:
import mlflow
import mlflow.sklearn
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Set up MLflow tracking URI
mlflow.set_tracking_uri("http://localhost:5000")

# Load dataset
iris = datasets.load_iris()
X = iris.data
y = iris.target

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Set experiment
mlflow.set_experiment("iris_test")

# Train and log model
with mlflow.start_run():
    model = LogisticRegression(max_iter=200)
    model.fit(X_train, y_train)
    
    # Predict and evaluate
    predictions = model.predict(X_test)
    acc = accuracy_score(y_test, predictions)

    # Log model and metrics
    mlflow.log_metric("accuracy", acc)
    mlflow.sklearn.log_model(model, "logistic_regression_model")

    print(f"Logged to MLflow with accuracy: {acc}")


2025/06/07 13:24:24 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Logged to MLflow with accuracy: 1.0
🏃 View run big-cow-401 at: http://localhost:5000/#/experiments/1/runs/826b09da3dc64f38a6eddfe36e4b9f8c
🧪 View experiment at: http://localhost:5000/#/experiments/1


In [3]:
from pathlib import Path

current_path = Path.cwd()
print(f"Current working directory: {current_path}")

Current working directory: c:\Users\dusad\Documents\Projects\agnei_consulting\mlutils\Notebooks


Git repository root: c:\Users\dusad\Documents\Projects\agnei_consulting\mlutils


### Attempt downloading with Kaggle

In [1]:
from mlutils.utils.kaggle import fetch_kaggle_dataset
from mlutils.utils.io import find_git_root
_dir = find_git_root() / "data"
# fetch_kaggle_dataset()

c:\Users\dusad\miniconda3\envs\agnei\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


100%|██████████| 172k/172k [00:00<00:00, 193kB/s]

Extracting files...


Path to dataset files: C:\Users\dusad\.cache\kagglehub\datasets\blastchar\telco-customer-churn\versions\1


ImportError: cannot import name 'fetch_kaggle_dataset' from 'mlutils.utils.kaggle' (C:\Users\dusad\Documents\Projects\agnei_consulting\mlutils\src\mlutils\utils\kaggle.py)